### ETL Australian user items

Este Archivo de trabajo, contiene las instrucciones necesarias para ejecutar la extración, transformación y carga del archivo "australian_users_items.json".

La intención inicial del documento, es mostrar el paso a paso de la extracción de la información, explicar y justificar cada una de las instrucciones ejecutadas para la transformación del archivo y brindar como resultado un archivo limpio en formato CSV.

In [ ]:
# Se importan las librerías necesarias para trabajar con archivos de tipo json
import pandas as pd # Se utiliza pandas para trabajar con dataframes
import json # Se utiliza json para la lectura de archivos en este formato
import numpy as np # Se utiliza numpy para ciertas operaciones matematicas en los dataframe

Haciendo una lectura previa del archivo, utilizando el editor de texto notepad++, podemos notar que el archivo no cumple con la estructura que debe de tener el formato json, es decir, que sus claves y valores, deben de venir especificados con comillas dobles, sin embargo, en el archivo están definidas con comillas simples.

Esto provoca que, tengamos que reemplazar estas comillas simples que delimitan las claves y valores, cuidando que no se sistituyan aquellas comillas simples existentes dentro de los valroes de tipo string como lo pueden ser las apostrofes.

In [12]:
# En este apartado, comenzamos con la lectura del archivo json y colocamos su contenido en un dataframe de pandas
Ruta_Json = '..\\..\\Inputs Originales\\australian_users_items.json' # Guardamos la ruta del archivo json a leer en una variable

# Se prcede a leer el archivo json con la intención de reemplazar las comillas simples
with open(Ruta_Json, 'r', encoding='utf-8') as archivo: # Se crea la variable archivo para leer el archivo
    contenido = archivo.read() # Se guarda el resultado de lectura del archivo en una variable
    # Para la sititución de las comillas simples que rodean a las claves y valores, se hará utilizando ciertos patrones que justo delimitan las claves y valores
    contenido = contenido.replace("', '", '","').replace("{'", '{"').replace("': '", '": "').replace("':", '":').replace(", '", ', "')  # Se aplica la sistitución de los patrones encontrados

In [13]:
# En este apartado, comenzamos con la lectura del archivo json y colocamos su contenido en un dataframe de pandas
Ruta_Json_Editado = '..\\..\\Inputs Originales\\australian_users_items_editado.json' # Guardamos la ruta del archivo json a leer en una variable

# Se procede a guardar el archivo editado como un nuevo archivo de tipo json
with open(Ruta_Json_Editado, 'w', encoding='utf-8') as archivo: # Se crea el archivo json editado
    archivo.write(contenido) # Se vacía el contenido dentro del archivo y se guarda

In [ ]:
# En este apartado, comenzamos con la lectura del archivo json editado y colocamos su contenido en un dataframe de pandas
Ruta_Json_Editado = '..\\..\\Inputs Originales\\australian_users_items_editado.json' # Guardamos la ruta del archivo json a leer en una variable

# Debido a que en el archivo hay varias líneas que no pueden ser decodificadas, utilizaremos una lista para ignorar la información que muestra error
datos_cargados = [] # Se crea una lista de datos cargados

# Leemos el archivo de json editado
with open(Ruta_Json_Editado, 'r', encoding='utf-8') as archivo: # Se lee el archivo en codificación utf8
    for linea in archivo: # Para cada línea existente en el archivo
        try: # Se capturan los errores que puedan existir al leer los datos
            datos = json.loads(linea) # Se cargan los datos de la línea leida
            datos_cargados.append(datos) # Se añade la fila leída a la lista datos
        except json.JSONDecodeError as e: # En caso de que exista un error en la carga
            continue # Se continua con el siguiente registro

# Se convierte el contenido trabajado previamente en un DataFrame
df_Users_Items = pd.DataFrame(datos_cargados) # Se asigna el DataFrame resultante'

In [ ]:
# Ahora que tenemos un Dataframe con la información del archivo, comenzamos a explorar el contenido y realizamos ciertos trabajos de limpieza

# Analizamos la estructura y contenido del dataframe
df_Users_Items.info() # Nos muestra la información general del dataframe
# Por ahora, tenemos un total de 5 columnas de tipo objeto

In [ ]:
# haciendo un análisis estadístico del Dataframe, veremos cuántos registros contiene, el total de vacíos y duplicados

print("Total de filas en el DataFrame: ", df_Users_Items.shape[0]) # Mostramos el total de registros del DataFrame 
# Se tiene un total de 88,146 registros dentro del archivo
print("Total de registros vacíos por columna: ", df_Users_Items.isnull().sum()) # Mostramos el total de registros vacíos

# Se reemplazan los valores vacíos descritos como 'null', 'None' con NaN en todo el DataFrame
df_Users_Items.replace(['', 'null', 'None'], np.nan, inplace=True) # Se guardel resultaddo en el DataFrame
# Se procede a eliminar aquellos registros que estén vacíos
df_Users_Items.dropna(inplace = True) # Se guarda el resultado en el mismo dataframe
print("Total de filas en el DataFrame sin registros vacíos: ", df_Users_Items.shape[0]) # Mostramos el total de registros del DataFrame 
# al no existir registros vacíos, no se elimina nada

In [ ]:
# Se nomraliza la columna de "items", aplanando sus valores y guardando el resultado en el mismo dataframe
df_Users_Items = pd.json_normalize(datos_cargados, record_path=['items'], meta=['steam_id','items_count','user_id','user_url'] ) # Se guarda el resultado en el mismo Dataframe
df_Users_Items = df_Users_Items.drop_duplicates(keep='first') # Se liminan duplicados del dataframe
df_Users_Items.shape[0] # Después de aplanar el archivo, nos damos cuenta que tenemos 4,935,329 registros

In [ ]:
# Debido a la cantidad de registros del DataFrame, se requiere hacer una limpieza que permita trabajar solo con los registros y columnas eficientes
filas_con_cero = (df_Users_Items['playtime_forever'] == 0).sum() # Contamos las filas con valor 0 en la columna 'hours_game'
print(f"Existen {filas_con_cero} filas con valor 0 en la columna 'hours_game'.") # se imprime el resultado
# Notamos que hay un total de 1,751,536 registros vacíos, es decir, un 35% de los registros resultan no ser representativos para nuestros análisis

In [ ]:
df_Users_Items = df_Users_Items[df_Users_Items['playtime_forever'] != 0] # Se eliminan del dataframe aquellos registros que contengan 0 en la columna playtime_foreves
df_Users_Items.shape[0] # Después de eliminar los registros sin juego del archivo, nos quedamos con 3,183,793 registros
df_Users_Items = df_Users_Items.drop(['item_name','playtime_2weeks','steam_id','items_count','user_url'], axis=1) # Se procede a eliminar aquellas columnas que no serían útiles para el análisis o que se pueden obtener desde otro csv

In [ ]:
# Se guarda el resultado de la limpieza en un nuevo archivo de tipo csv
df_Users_Items.to_csv('..\\..\\Archivos Limpios\\australian_users_items_limpio.csv', index=False)

# Por ahora, se ha terminado con la limpieza de la información del archivo users_items